In [2]:
import pandas as pd
import numpy as np

In [225]:
data_age = pd.read_csv("id_age_train.csv", sep=",")
data_vitals = pd.read_csv("id_time_vitals_train.csv", sep=",")
data_labels = pd.read_csv("id_label_train.csv", sep=",").set_index('ID')
data_labs = pd.read_csv("id_time_labs_train.csv", sep = ",")
data_timeseries = pd.merge(data_labs, data_vitals)
patients = list(data_timeseries['ID'])
patient_ids = list(data_timeseries['ID'].unique())
labels_zeros = np.zeros(len(patients))
labels = np.zeros(len(patients))
for p in patient_ids:
    if int(data_labels.ix[p]['LABEL'])==1:
        #labels[data_timeseries[data_timeseries['ID']==p].index[-1]] = 1
        data_id = data_timeseries[data_timeseries['ID']==p]
        indices = data_id[data_id['ICU']==1].index
        num = len(indices)
        for i in indices:
            labels_zeros[i]=1
data_timeseries['LABEL'] = labels_zeros
data = pd.merge( data_age, data_timeseries, on='ID')

In [59]:
data.head()

,ID,AGE,TIME,L1,L2,L3,L4,L5,L6,L7,...,L24,L25,V1,V2,V3,V4,V5,V6,ICU,LABEL
0,1,42,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,86,49,70,NaN,87,NaN,0,0
1,1,42,4320,6.60759,114.044116,128.393872,NaN,NaN,11.327002,NaN,...,NaN,NaN,NaN,NaN,70,NaN,NaN,NaN,0,0
2,1,42,5646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,91,58,NaN,NaN,NaN,96.6,1,0
3,1,42,5703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,140,73,91,32,NaN,NaN,1,0
4,1,42,6342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,139,90,107,29,101,NaN,1,0


l3max, l5max, l7!!!, l8max, l16, l18max?, l19max?, l20max!!!, l22max, l23max, v3max?   

In [226]:
data.loc[445378,'L20'] = 20.5267

In [61]:
data['ID'] = data['ID'].astype('int')
data['AGE'] = data['AGE'].astype('int')
data['TIME'] = data['TIME'].astype('int')
data['ICU'] = data['ICU'].astype('int')
data['LABEL'] = data['LABEL'].astype('int')

In [227]:
data_grouped = data.groupby(['ID','TIME']).sum()
normal = {attribute: data[attribute].mean() for attribute in data.columns}
patient_ids = data['ID'].unique()
for patient in patient_ids:
    data_grouped.loc[patient, 0] = data_grouped.loc[patient, 0].fillna(normal)
data_filled = data_grouped.reset_index().fillna(method='ffill')

In [67]:
data_filled.head()

,ID,TIME,AGE,L1,L2,L3,L4,L5,L6,L7,...,L24,L25,V1,V2,V3,V4,V5,V6,ICU,LABEL
0,1,0,42,7.367832,43.743643,121.660822,137.729164,4.053076,25.617869,34.883442,...,37.891432,1.970802,86,49,70,20.426097,87,98.24268,0,0
1,1,4320,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,...,37.891432,1.970802,86,49,70,20.426097,87,98.24268,0,0
2,1,5646,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,...,37.891432,1.970802,91,58,70,20.426097,87,96.60000,1,0
3,1,5703,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,...,37.891432,1.970802,140,73,91,32.000000,87,96.60000,1,0
4,1,6342,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,...,37.891432,1.970802,139,90,107,29.000000,101,96.60000,1,0


In [232]:
data_np = data_filled.as_matrix()

In [300]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', warm_start=True)

from sklearn.cross_validation import train_test_split
train, test = train_test_split(data_np, test_size=0.3, random_state = 21)

In [285]:
train_df = pd.DataFrame(train, columns=data.columns)
train_df.sort(['ID','AGE'],inplace=True)
train = train_df.as_matrix()

In [286]:
from sklearn.preprocessing import scale

In [287]:
train_feats = train[:,1:-2]
train_feats = scale(train_feats)

In [288]:
train_feats

array([[ -3.68436677e-01,  -1.29874261e+00,  -3.87158341e+00, ...,
         -1.53727901e-03,  -2.76543264e+00,  -1.35434270e+00],
       [ -3.68416035e-01,  -1.29874261e+00,  -3.87158341e+00, ...,
          1.95838083e+00,  -2.76543264e+00,  -1.35434270e+00],
       [ -3.68184632e-01,  -1.29874261e+00,  -3.87158341e+00, ...,
          1.45036254e+00,   8.76398864e-01,  -1.35434270e+00],
       ..., 
       [ -2.63262658e-01,  -7.07328848e-01,   3.83987826e-01, ...,
         -5.81710622e-01,   9.60063994e-02,  -5.74038068e-03],
       [ -2.62168654e-01,  -7.07328848e-01,   3.83987826e-01, ...,
         -5.81710622e-01,  -1.20464771e+00,  -5.74038068e-03],
       [ -2.58572308e-01,  -7.07328848e-01,   3.83987826e-01, ...,
         -5.81710622e-01,  -9.44516887e-01,  -2.43729026e-01]])

In [301]:
clf.fit(train_feats, train[:,-1])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=True)

In [290]:
sum(train[:,-1])

48075.0

In [291]:
test_df = pd.DataFrame(test, columns=data.columns)
test_df.sort(['ID','AGE'], inplace=True)
test_df.set_index('ID', inplace=True)
test_ids = set(test_df.index)

In [284]:
test_df.head()

,AGE,TIME,L1,L2,L3,L4,L5,L6,L7,L8,...,L24,L25,V1,V2,V3,V4,V5,V6,ICU,LABEL
ID,,,,,,,,,,,,,,,,,,,,,
1,0,42,7.367832,43.743643,121.660822,137.729164,4.053076,25.617869,34.883442,1.908826,...,37.891432,1.970802,86,49,70,20.426097,87,98.24268,0,0
1,4320,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,1.908826,...,37.891432,1.970802,86,49,70,20.426097,87,98.24268,0,0
1,6609,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,1.908826,...,37.891432,1.970802,152,75,109,30.000000,101,96.60000,1,1
1,6894,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,1.908826,...,37.891432,1.970802,140,79,84,30.000000,98,96.60000,1,1
1,10213,42,6.607590,114.044116,128.393872,137.729164,4.053076,11.327002,34.883442,1.908826,...,37.891432,1.970802,134,68,105,31.000000,99,95.20000,1,1


In [182]:
test_1 = test_df.ix[1]
test_1_np = np.atleast_2d(test_1.as_matrix())
icu_indices = test_1_np[:,-2]==1
icu_data = test_1_np[icu_indices,:]
predictions = []
for ind in range(icu_data.shape[0]):
    print ind
    partial_data = scale(icu_data[:ind+1,:-2])
    print partial_data
    #if ind>2:
     #   clf.partial_fit(partial_data[:-1,:], predictions[:ind])
    pred = clf.predict(partial_data[-1,:])
    print 'pred',pred
    print 'actual', icu_data[ind,-1]
    predictions.append(pred)

0
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
pred [ 0.]
actual 1.0
1
[[ 1.  0.  1. -1. -1. -1. -1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  1. -1.  0. -1. -1.]
 [-1.  0. -1.  1.  1.  1.  1. -1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  1.  0.  0.  0.  0.  0. -1. -1.  1.  0.  1.  1.]]
pred [ 1.]
actual 1.0
2
[[ 1.31805048  0.          1.41421356 -1.41421356 -1.41421356  0.20758882
  -1.00156534  1.41421356 -0.70710678  0.70710678  0.70710678  0.94161359
   0.70710678 -0.70710678  0.          0.          0.70710678 -0.70710678
   0.          0.70710678  0.         -1.41421356 -0.70710678  0.70710678
   0.          0.          0.70710678  0.          1.08908156 -1.07973422
   0.70710678 -1.41022048 -1.3786012 ]
 [-0.21511398  0.         -0.70710678  0.70710678  0.70710678  1.10768409
   1.36545032 -0.70710678 -0.70710678  0.70710678  0.7

In [302]:
final_preds = []
final_answers =[]
for id in test_ids:
    test_id = test_df.ix[id]
    test_id_np = test_id.as_matrix()
    test_id_np = np.atleast_2d(test_id_np)
    icu_indices = np.nonzero(test_id_np[:,-2])[0]
    icu_data = test_id_np[icu_indices]
    prev_prediction = 0
    final_answers.append(data_labels.ix[int(id)]['LABEL'])
    flag = 0
    final=0
    icui=0
    for ind in icu_indices:
        partial_data = test_id_np[:ind+1,:-2]
        partial_data_feats = partial_data
        partial_data_feats = scale(partial_data_feats)
        if icui==0 and ind>0:
            clf.partial_fit(partial_data_feats[:-1,:], np.zeros((partial_data_feats[:-1].shape[0],)))
        if icui>0:
            if ind - icu_indices[icui-1]>1 :
                clf.partial_fit(partial_data_feats[icu_indices[icui-1]+1:ind,:], np.zeros((partial_data_feats[icu_indices[icui-1]+1:ind,:].shape[0],)))
            prev_icu = np.atleast_2d(partial_data_feats[icu_indices[icui-1],:])
            clf.partial_fit(prev_icu, prev_prediction)
        prediction = clf.predict(partial_data_feats[-1,:])
        prev_prediction = prediction
        if flag==0 and int(prediction[0])==1:
            final=1
            flag+=1
        icui+=1
    final_preds.append(final)

In [35]:
from sklearn.metrics import confusion_matrix

In [303]:
cm = confusion_matrix(final_answers, final_preds)

In [304]:
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

In [305]:
float(TN)/(TN+FP)

1.0

In [306]:
float(TP)/(TP+FN)

0.0

In [307]:
cm

array([[3342,    0],
       [ 244,    0]])

In [315]:
def get_data():
    data_age = pd.read_csv("id_age_train.csv", sep=",")
    data_vitals = pd.read_csv("id_time_vitals_train.csv", sep=",")
    data_labels = pd.read_csv("id_label_train.csv", sep=",").set_index('ID')
    data_labs = pd.read_csv("id_time_labs_train.csv", sep = ",")
    data_timeseries = pd.merge(data_labs, data_vitals)
    patients = list(data_timeseries['ID'])
    patient_ids = list(data_timeseries['ID'].unique())
    labels_zeros = np.zeros(len(patients))
    labels = np.zeros(len(patients))
    for p in patient_ids:
        if int(data_labels.ix[p]['LABEL'])==1:
            #labels[data_timeseries[data_timeseries['ID']==p].index[-1]] = 1
            data_id = data_timeseries[data_timeseries['ID']==p]
            indices = data_id[data_id['ICU']==1].index
            num = len(indices)
            for i in indices:
                labels_zeros[i]=1
    data_timeseries['LABEL'] = labels_zeros
    data = pd.merge( data_age, data_timeseries, on='ID')
    data.loc[445378,'L20'] = 20.5267
    return data

In [312]:
def preprocess(data):
    data_grouped = data.groupby(['ID','TIME']).sum()
    normal = {attribute: data[attribute].mean() for attribute in data.columns}
    patient_ids = data['ID'].unique()
    for patient in patient_ids:
        data_grouped.loc[patient, 0] = data_grouped.loc[patient, 0].fillna(normal)
    data_filled = data_grouped.reset_index().fillna(method='ffill')
    data_np = data_filled.as_matrix()
    return data_np, data_filled.columns

In [314]:
def find_scores(data_np, column_names,classifer, **kwargs):
    clf = classifier(**kwargs)
    train, test = train_test_split(data_np, test_size=0.3, random_state = 21)
    train_df = pd.DataFrame(train, columns=column_names)
    train_df.sort(['ID','TIME'],inplace=True)
    train = train_df.as_matrix()
    train_feats = train[:,1:-2]
    train_feats = scale(train_feats)
    clf.fit(train_feats, train[:,-1])
    test_df = pd.DataFrame(test, columns=columns_names)
    test_df.sort(['ID','AGE'], inplace=True)
    test_df.set_index('ID', inplace=True)
    test_ids = set(test_df.index)
    final_preds = []
    final_answers =[]
    for id in test_ids:
        test_id = test_df.ix[id]
        test_id_np = test_id.as_matrix()
        test_id_np = np.atleast_2d(test_id_np)
        icu_indices = np.nonzero(test_id_np[:,-2])[0]
        prev_prediction = 0
        final_answers.append(data_labels.ix[int(id)]['LABEL'])
        flag = 0
        final=0
        icui=0
        for ind in icu_indices:
            partial_data = test_id_np[:ind+1,:-2]
            partial_data_feats = scale(partial_data)
            if icui==0 and ind>0:
                clf.partial_fit(partial_data_feats[:-1,:], np.zeros((partial_data_feats[:-1].shape[0],)))
            if icui>0:
                if ind - icu_indices[icui-1]>1 :
                    clf.partial_fit(partial_data_feats[icu_indices[icui-1]+1:ind,:], np.zeros((partial_data_feats[icu_indices[icui-1]+1:ind,:].shape[0],)))
                prev_icu = np.atleast_2d(partial_data_feats[icu_indices[icui-1],:])
                clf.partial_fit(prev_icu, prev_prediction)
            prediction = clf.predict(partial_data_feats[-1,:])
            prev_prediction = prediction
            if flag==0 and int(prediction[0])==1:
                final=1
                flag+=1
            icui+=1
        final_preds.append(final)
    cm = confusion_matrix(final_answers, final_preds)
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TP = cm[1][1]
    specificity = float(TN)/(TN+FP)
    sensitivity = float(TP)/(TP+FN)
    return specificity, sensitivity

In [316]:
# whatever params you give after classifier become the params for it. eg main(SGDClassifier, loss='log', warm_start=True)
def main(classifier, **kwargs):
    data = get_data()
    data_np,column_names = preprocess(data)
    specificity, sensitivity = find_scores(data_np, column_names, classifier, **kwargs)